<a href="https://colab.research.google.com/github/kavsraj/Applied_Datascience_Capstone_Project/blob/master/Segmenting_%26_Clustering_Neighborhoods_in_Toronto_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import libraries
import pandas as pd
!pip -q install lxml

!pip install requests
import requests 
import io

#read the data from the url and set the index 
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=890001695"
df = pd.read_html(url)[0]

# Ignore cells with a borough that is 'Not assigned'.
df = df[df.Borough != 'Not assigned']
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(x)).to_frame().reset_index()

# Neighborhoods separated with a comma
for index,row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
      row['Neighbourhood'] = row['Borough']

#we need to get the latitude and the longitude coordinates of each neighborhood in order to utilize the Foursquare location data.
url1="http://cocl.us/Geospatial_data"
req=requests.get(url1).content
dec=pd.read_csv(io.StringIO(req.decode('utf-8')))

# rename the first column to allow merging dataframes on Postcode
dec.columns = ['Postcode', 'Latitude', 'Longitude']
df = pd.merge(dec, df, on='Postcode')

# reorder column names and show the dataframe
df = df[['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]
df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437
